# Getting Started with BentoML

[BentoML](http://bentoml.ai) is an open source framework for serving and deploying machine learning models. It provides high-level APIs for defining a prediction service and packaging trained models, source code, dependencies, and configurations into a production-system-friendly format that is ready for production deployment.

This is a quick tutorial on how to use BentoML to create a prediction service with a trained sklearn model, serving the model via a REST API server and deploy it to [AWS Lambda](https://aws.amazon.com/lambda/) as a serverless endpoint.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=guides&ea=bentoml-quick-start-guide&dt=bentoml-quick-start-guide)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

BentoML requires python 3.6 or above, install via `pip`:

In [ ]:
# Install BentoML
!pip install bentoml

# Also install scikit-learn, we will use a sklean model as an example
!pip install pandas sklearn

Let's get started with a simple scikit-learn model as an example:

In [2]:
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## Creating a Prediction Service with BentoML


The first step of creating a prediction service with BentoML, is to write a prediction service class inheriting from bentoml.BentoService, and declaratively listing the dependencies, model artifacts and writing your service API call back function. Here is what a simple prediction service looks like:

In [3]:
%%writefile iris_classifier.py
from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler

@artifacts([SklearnModelArtifact('model')])
@env(pip_dependencies=["scikit-learn"])
class IrisClassifier(BentoService):

    @api(DataframeHandler)
    def predict(self, df):
        return self.artifacts.model.predict(df)

Writing iris_classifier.py


The `bentoml.api` and `DataframeHandler` here tells BentoML, that following by it, is
the service API callback function, and `pandas.Dataframe` is its expected input format.

The `bentoml.env` decorator allows user to specify the dependencies and environment 
settings for this prediction service. Here we are creating the prediction service based
on a scikit learn model, so we add it to the list of pip dependencies.


Last but not least, `bentoml.artifact` declares the required trained model to be bundled
with this prediction service. Here it is using the built-in `SklearnModelArtifact` and
simply naming it 'model'. BentoML also provide model artifact for other frameworks such
as `PytorchModelArtifact`, `KerasModelArtifact`, `FastaiModelArtifact`, and
`XgboostModelArtifact` etc.


## Saving a versioned BentoService bundle

In [4]:
# 1) import the custom BentoService defined above
from iris_classifier import IrisClassifier

# 2) `pack` it with required artifacts
svc = IrisClassifier.pack(model=clf)

# 3) save BentoSerivce to a BentoML bundle
saved_path = svc.save()

[2020-01-21 11:27:10,327] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-21 11:27:10,731] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.5.8+14.g67ecb46.dirty
creating BentoML-0.5.8+14.g67ecb46.dirty/BentoML.egg-info
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/artifact
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/bundler
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/cli
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/clipper
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/configuration
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/deployment
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/deployment/sagemaker
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/handlers
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/migrations
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/migrations/versions
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/repository
creating BentoML-0.5.8+14.g67ecb46.dirty/bentoml/server
creat

copying bentoml/proto/__init__.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/proto/deployment_pb2.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/proto/repository_pb2.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/proto/status_pb2.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.5.8+14.g67ecb46.dirty/bentoml/ser

_That's it._ You've just created a BentoService SavedBundle, it's a versioned file archive that is ready for production deployment. It contains the BentoService you defined, as well as the packed trained model artifacts, pre-processing code, dependencies and other configurations in a single file directory.

## Model Serving via REST API

From a BentoService SavedBundle, you can start a REST API server by providing the file path to the saved bundle:

In [5]:
# Note that REST API serving **does not work in Google Colab** due to unable to access Colab's VM
!bentoml serve {saved_path}

[2020-01-21 11:27:29,999] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-21 11:27:30,013] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+14.g67ecb46.dirty
[2020-01-21 11:27:30,597] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

#### View documentations for REST APIs

The REST API server provides a simply web UI for you to test and debug. If you are running this command on your local machine, visit http://127.0.0.1:5000 in your browser and try out sending API request to the server.

![BentoML API Server Web UI Screenshot](https://raw.githubusercontent.com/bentoml/BentoML/master/guides/quick-start/bento-api-server-web-ui.png)

#### Send prediction request to REST API server

You can also send prediction request with `curl` from command line:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
localhost:5000/predict
```

Or with `python` and `request` library:
```python
import requests
response = requests.post("http://127.0.0.1:5000/predict", json=[[5.1, 3.5, 1.4, 0.2]])
print(response.text)
```



## Containerize REST API server with Docker


The BentoService SavedBundle is structured to work as a docker build context, that can
be directed used to build a docker image for API server. Simply use it as the docker
build context directory:

In [6]:
!cd {saved_path} && docker build -t iris-classifier .

Sending build context to Docker daemon  1.011MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Running in bbab57ee8a88
Removing intermediate container bbab57ee8a88
 ---> d3cfb80f9fc7
Step 3/12 : EXPOSE 5000
 ---> Running in f953dcb79abc
Removing intermediate container f953dcb79abc
 ---> 3d1f997a1d9c
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Running in c2e99bcf2947
+ apt-get update
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [172 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7908 kB]
Fetched 8316 kB in 3s

Unpacking libitm1:amd64 (8.3.0-6) ...
Selecting previously unselected package libatomic1:amd64.
Preparing to unpack .../15-libatomic1_8.3.0-6_amd64.deb ...
Unpacking libatomic1:amd64 (8.3.0-6) ...
Selecting previously unselected package libasan5:amd64.
Preparing to unpack .../16-libasan5_8.3.0-6_amd64.deb ...
Unpacking libasan5:amd64 (8.3.0-6) ...
Selecting previously unselected package liblsan0:amd64.
Preparing to unpack .../17-liblsan0_8.3.0-6_amd64.deb ...
Unpacking liblsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libtsan0:amd64.
Preparing to unpack .../18-libtsan0_8.3.0-6_amd64.deb ...
Unpacking libtsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libubsan1:amd64.
Preparing to unpack .../19-libubsan1_8.3.0-6_amd64.deb ...
Unpacking libubsan1:amd64 (8.3.0-6) ...
Selecting previously unselected package libmpx2:amd64.
Preparing to unpack .../20-libmpx2_8.3.0-6_amd64.deb ...
Unpacking libmpx2:amd64 (8.3.0-6) ...
Selecting previously unselected packag

mkl_fft-1.0.15       | 154 KB    | ########## | 100% 
numpy-1.18.1         | 5 KB      | ########## | 100% 
scipy-1.3.2          | 13.9 MB   | ########## | 100% 
intel-openmp-2019.4  | 729 KB    | ########## | 100% 
certifi-2019.11.28   | 153 KB    | ########## | 100% 
conda-4.8.1          | 2.8 MB    | ########## | 100% 
blas-1.0             | 6 KB      | ########## | 100% 
libgfortran-ng-7.3.0 | 1006 KB   | ########## | 100% 
mkl_random-1.1.0     | 321 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container 176267e5ab58
 ---> 504b4d11f51a
Step 6/12 : COPY . /bento
 ---> 66d1e244e3ca
Step 7/12 : WORKDIR /bento
 ---> Running in 746540e44412
Removing intermediate container 746540e44412
 ---> 62112a52592a
Step 8/12 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 4f5b9aca959a
Solving environment: ...working... done

python-3.7.2        

  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.12-cp37-cp37m-linux_x86_64.whl size=1219999 sha256=dc002859dc6770f36ff2a652cb0d0e2f99da62efaf2d267afd583ef3bb9180ff
  Stored in directory: /root/.cache/pip/wheels/ee/33/44/0788a6e806866ae2e246d5cd841d07498a46bcb3f3c42ea5a4
  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=a9d82f90e0c0b1b6aee24c4607487c69b605d3b8d0ec25ddefcfe79b019a4ed8
  Stored in directory: /root/.cache/pip/wheels/97/f7/a1/752e22bb30c1cfe38194ea0070a5c66e76ef4d06ad0c7dc401
  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23274 sha256=4941cdceff4c3a91051c8336692de1eede19460b01538d91d34a1e2aba0f91ca
  Stored in directory: /root/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for prometheus-client: filename=prometheus_client-0.7.1-cp37-none-any.whl size=41402 sha256=1f0fef77d9a6e56eb0b6d081515b92c4e169037c3ba0ee7007f3147e262f762c
 

  Created wheel for BentoML: filename=BentoML-0.5.8+14.g67ecb46.dirty-cp37-none-any.whl size=504368 sha256=7fbaaeb21173ecff3d19ba3555fe1e65fe71349a00547f6f826cf32911975ce9
  Stored in directory: /root/.cache/pip/wheels/36/48/47/c79863a4fa74704bbbc25fd4b8f5c2c97651cf3227018a97a1
Successfully built BentoML
  Found existing installation: BentoML 0.5.8
    Uninstalling BentoML-0.5.8:
      Successfully uninstalled BentoML-0.5.8
Removing intermediate container 1f3c968c4f6a
 ---> 214ca9af86a7
Step 11/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in e4a67a94f37c
Removing intermediate container e4a67a94f37c
 ---> 2df40389cef8
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in df292cb29c8a
Removing intermediate container df292cb29c8a
 ---> de209d021ad3
Successfully built de209d021ad3
Successfully tagged iris-classifier:latest


Note that `docker` is __note available in Google Colab__, download the notebook, ensure docker is installed and try it locally.

Next, you can `docker push` the image to your choice of registry for deployment,
or run it locally for development and testing:

In [7]:
!docker run -p 5000:5000 iris-classifier

[2020-01-21 19:30:32,968] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-01-21 19:30:33 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-01-21 19:30:33 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-01-21 19:30:33 +0000] [1] [INFO] Using worker: sync
[2020-01-21 19:30:33 +0000] [11] [INFO] Booting worker with pid: 11
[2020-01-21 19:30:33 +0000] [12] [INFO] Booting worker with pid: 12
[2020-01-21 19:30:33 +0000] [13] [INFO] Booting worker with pid: 13
[2020-01-21 19:30:33,406] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_de

## Load saved BentoService

`bentoml.load` is the enssential API for loading a Bento into your
python application:

In [8]:
import bentoml
import pandas as pd

bento_svc = bentoml.load(saved_path)

# Test loaded bentoml service:
bento_svc.predict([X[0]])

[2020-01-21 11:30:55,579] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-21 11:30:55,591] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+14.g67ecb46.dirty
[2020-01-21 11:30:55,593] WARNING - Module `iris_classifier` already loaded, using existing imported module.
[2020-01-21 11:30:55,610] WARNING - BentoML local changes detected - Local BentoML repository including all code

memmap([0])

## Distribute BentoML SavedBundle as PyPI package


The BentoService SavedBundle is pip-installable and can be directly distributed as a
PyPI package if you plan to use the model in your python applications. You can install
it as as a system-wide python package with `pip`:

In [ ]:
!pip install {saved_path}

In [10]:
# Your bentoML model class name will become packaged name
import IrisClassifier

installed_svc = IrisClassifier.load()
installed_svc.predict([X[0]])

memmap([0])

This also allow users to upload their BentoService to pypi.org as public python package
or to their organization's private PyPi index to share with other developers.

`cd {saved_path} & python setup.py sdist upload`

*You will have to configure ".pypirc" file before uploading to pypi index.
    You can find more information about distributing python package at:
    https://docs.python.org/3.7/distributing/index.html#distributing-index*


# Model Serving via CLI

`pip install {saved_path}` also installs a CLI tool for accessing the BentoML service, print CLI help document with `--help`:


In [11]:
!IrisClassifier --help

Usage: IrisClassifier [OPTIONS] COMMAND [ARGS]...

  BentoML CLI tool

Options:
  -q, --quiet         Hide process logs and only print command results
  --verbose, --debug  Print verbose debugging information for BentoML
                      developer
  --version           Show the version and exit.
  --help              Show this message and exit.

Commands:
  <API_NAME>      Run API function
  info            List APIs
  open-api-spec   Display OpenAPI/Swagger JSON specs
  serve           Start local rest server
  serve-gunicorn  Start local gunicorn server


Printing more information about this ML service with `info` command:

In [ ]:
!IrisClassifier info

You can also print help and docs on individual commands:

In [ ]:
!IrisClassifier predict --help

Each service API you defined in the BentoService will be exposed as a CLI command with the same name as the API function:

In [12]:
!IrisClassifier predict --input='[[5.1, 3.5, 1.4, 0.2]]'

[2020-01-21 11:31:43,990] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-21 11:31:43,998] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+14.g67ecb46.dirty
[2020-01-21 11:31:44,000] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

BentoML cli also supports reading input data from `csv` or `json` files, in either local machine or remote HTTP/S3 location:

In [13]:
# Writing test data to a csv file
pd.DataFrame(iris.data).to_csv('iris_data.csv', index=False)

# Invoke predict from command lien
!IrisClassifier predict --input='./iris_data.csv'

[2020-01-21 11:31:50,523] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.5.8. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-01-21 11:31:50,532] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.5.8,  but loading from BentoML version 0.5.8+14.g67ecb46.dirty
[2020-01-21 11:31:50,535] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

Alternatively, you can also use the `bentoml` cli to load and run a BentoML service archive without installing it:

In [ ]:
!bentoml info {saved_path}

In [14]:
!bentoml predict {saved_path} --input='[[5.1, 3.5, 1.4, 0.2]]'

[0]


# Deploy REST API server to the cloud


BentoML has a built-in deployment management tool called YataiService. YataiService can
be deployed separately to manage all your teams' trained models, BentoService bundles,
and active deployments in the cloud or in your own kubernetes cluster. You can also
create simple model serving deployments with just the BentoML cli, which launches a
local YataiService backed by SQLite database on your machine.

Now let's deploy the IrisClassifier to [AWS Lambda](https://aws.amazon.com/lambda/) as
a serverless endpoint.

First you need to install the `aws-sam-cli` package, which is required by BentoML
to work with AWS Lambda deployment:

```
    pip install -U aws-sam-cli==0.31.1
```


You will also need to configure your AWS account and credentials if you don't have
it configured on your machine. You can do this either
[via environment variables](https://docs.aws.amazon.com/sdk-for-java/v1/developer-guide/setup-credentials.html)
or through the `aws configure` command: install `aws` cli command via
`pip install awscli` and follow
[detailed instructions here](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html#cli-quick-configuration).

Now you can run the `bentoml lambda deploy` command, to create a AWS Lambda deployment,
hosting the BentService you've created:

In [15]:
!bentoml lambda deploy quick-start-guide-deployment -b IrisClassifier:{svc.version} 

Deploying Lambda deployment -[2020-01-21 11:32:14,793] INFO - Building lambda project
/[2020-01-21 11:33:38,233] INFO - Packaging AWS Lambda project at /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-fhkd3sst ...
/[2020-01-21 11:34:00,383] INFO - Deploying lambda project
|[2020-01-21 11:34:54,075] INFO - ApplyDeployment (quick-start-guide-deployment, namespace bobo) succeeded
Successfully created AWS Lambda deployment quick-start-guide-deployment
{
  "namespace": "bobo",
  "name": "quick-start-guide-deployment",
  "spec": {
    "bentoName": "IrisClassifier",
    "bentoVersion": "20200121112710_DA5806",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://3teiopjoba.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-bobo-quick-start-guide-deploy

Here the 'quick-starrt-guide-deployment' is the deployment name, you can reference the deployment by this name and query its status. For example, to get current deployment status:

In [16]:
!bentoml lambda get quick-start-guide-deployment

{
  "namespace": "bobo",
  "name": "quick-start-guide-deployment",
  "spec": {
    "bentoName": "IrisClassifier",
    "bentoVersion": "20200121112710_DA5806",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://3teiopjoba.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-bobo-quick-start-guide-deployment-2bac57"
    },
    "timestamp": "2020-01-21T19:35:58.644630Z"
  },
  "createdAt": "2020-01-21T19:32:10.241484Z",
  "lastUpdatedAt": "2020-01-21T19:32:10.241525Z"
}


To send request to your AWS Lambda deployment, grab the endpoint URL from the json output above:

In [17]:
!curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[5.1, 3.5, 1.4, 0.2]]' \
https://3teiopjoba.execute-api.us-west-2.amazonaws.com/Prod/predict














[0]

And to delete an active deployment:

In [18]:
!bentoml lambda delete quick-start-guide-deployment

Successfully deleted AWS Lambda deployment "quick-start-guide-deployment"


BentoML by default stores the deployment metadata on the local machine. For team settings, we recommend hosting a shared BentoML Yatai server for your entire team to track all BentoService saved bundle and deployments they've created in a central place.

# Summary

This is what it looks like when using BentoML to serve and deploy a model, as a prediction service running in the cloud. BentoML also supports many other Machine Learning frameworks, as well as many other deployment platforms. You can find more BentoML example notebooks [here](https://github.com/bentoml/BentoML#examples).